In [21]:
import argparse
import time
import math
import random
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image

from model import DRNetModel
from dataset import BasicDataset

from pathlib import Path
import utils

# Add base path to import dir for importing datasets
import os, sys, inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
from datasets import tub, shanghaitech
import cv2

In [2]:
val_frames = shanghaitech.load_all_frames('../data/ShanghaiTech/part_A_final/test_data', load_labeling=False)

In [4]:
dataset = BasicDataset(val_frames, 'flex', 64, False)

In [37]:
import numpy as np

In [62]:
total = []

for frame in val_frames:
    density_np = frame.get_density('flex')
    s = density_np.shape
    
    cv_dense = cv2.resize(density_np, (int(s[1] / 8), int(s[0] / 8)), interpolation=cv2.INTER_CUBIC) * 64
    
    torch_dense = torch.FloatTensor(density_np)
    torch_dense.unsqueeze_(0)
    torch_dense.unsqueeze_(0)
    torch_dense = F.interpolate(input=torch_dense,
                                     size=(int(s[0] / 8), int(s[1] / 8)),
                                     mode='bilinear', align_corners=True) * 64
    torch_dense.squeeze_()
    torch_dense = torch_dense.numpy()

    total.append(abs(torch_dense.max()-cv_dense.max()))

In [63]:
sum(total)

103.59787707030773

In [53]:
torch_dense = torch.FloatTensor(density_np)
torch_dense.unsqueeze_(0)
torch_dense.unsqueeze_(0)
torch_dense = F.interpolate(input=torch_dense,
                                 size=(int(s[0] / 8), int(s[1] / 8)),
                                 mode='bilinear', align_corners=False) * 64
torch_dense.squeeze_()
save_image(torch_dense, 'test.png')
print(torch_dense.shape)
torch_dense = torch_dense.numpy()
print(torch_dense.shape)

torch.Size([60, 80])
(60, 80)


In [43]:
print(np.sum(cv_dense-torch_dense))

0.10579192
